In [12]:
import torch  # Import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Training

In [15]:
# --- Training Code ---

# Import necessary libraries
import pandas as pd  # For data manipulation
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer  # For using the GPT-2 model and training
import torch  # For deep learning operations
import evaluate  # For evaluating the model
from sklearn.model_selection import train_test_split  # For splitting data

# Set the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the dataset from a CSV file
df = pd.read_csv("youtube_data.csv")

# Convert non-string values in 'video_title' column to strings
df['video_title'] = df['video_title'].fillna('').astype(str)

# Split data into training and evaluation sets (80% train, 20% eval)
train_df, eval_df = train_test_split(df, test_size=0.2) #random_state=42 

# Extract video titles for training and evaluation
train_titles = train_df['video_title'].tolist()
eval_titles = eval_df['video_title'].tolist()

# Initialize tokenizer and model
# model_name = "gpt2"  # Jika mau menggunakan gpt2 (model dasar)
model_name = "EdwardFang09/QuestionableYouTubeVideoTitleV1"  # Path to your initial fine-tuned model
tokenizer = AutoTokenizer.from_pretrained(model_name)  # Load the tokenizer

# Add a padding token (often set to EOS token for language models)
tokenizer.pad_token = tokenizer.eos_token 

# Load the pre-trained GPT-2 model
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to(device)  # Move the model to the appropriate device

# Prepare the dataset for training
class TitlesDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings  # Store the tokenized encodings

    def __getitem__(self, idx):
        # Retrieve encoded data and create labels (targets)
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()  # Labels are the same as input IDs for language modeling
        return item

    def __len__(self):
        return len(self.encodings.input_ids)  # Return the length of the dataset

# Tokenize the titles for training and evaluation
train_encodings = tokenizer(train_titles, truncation=True, padding=True)  # Truncate long titles and pad shorter ones
eval_encodings = tokenizer(eval_titles, truncation=True, padding=True) 

# Create dataset objects
train_dataset = TitlesDataset(train_encodings)
eval_dataset = TitlesDataset(eval_encodings)

# Define a metric (BLEU in this case)
metric = evaluate.load('bleu')  # Load the BLEU metric for evaluation

# Define a function to compute the BLEU score
def compute_metrics(pred):
    labels_ids = pred.label_ids  # Get the true labels
    pred_ids = pred.predictions  # Get the predicted labels

    # Decode the predicted IDs to text
    pred_str = [
        tokenizer.decode(ids, skip_special_tokens=True) 
        for ids in pred_ids 
        if all(isinstance(x, int) and x >= 0 for x in ids)  # Check for valid integer IDs
    ]

    # Replace -100 (ignore index) with EOS token ID in true labels
    labels_ids[labels_ids == -100] = tokenizer.eos_token_id 
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)  # Decode true labels

    # Check if predictions and labels are valid and have the same length
    if pred_str and label_str and len(pred_str) == len(label_str):
        # Compute the BLEU score
        result = metric.compute(predictions=pred_str, references=[[l] for l in label_str])
        return {
            "bleu": result["bleu"],
        }
    else:
        # Handle cases with empty or mismatched predictions and labels
        print("Warning: Empty predictions or labels encountered, or mismatched lengths.")
        return {"bleu": 0}  # Or another appropriate default value

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",  # Directory to save training outputs
    num_train_epochs=30,  # Number of training epochs
    per_device_train_batch_size=4,  # Batch size per device
    save_steps=1000,  # Save checkpoints every 1000 steps
    logging_dir="./logs",  # Directory for training logs
    fp16=True,  # Enable mixed precision training (if supported)
    evaluation_strategy="epoch",  # Evaluate the model at the end of each epoch
    save_strategy="epoch",  # Save the model at the end of each epoch
    load_best_model_at_end=True,  # Load the best performing model at the end of training
    metric_for_best_model="bleu",  # Use the BLEU score to select the best model
)

# Define the Trainer
trainer = Trainer(
    model=model,  # The GPT-2 model to be fine-tuned
    args=training_args,  # Training arguments
    train_dataset=train_dataset,  # Training dataset
    eval_dataset=eval_dataset,  # Evaluation dataset
    compute_metrics=compute_metrics,  # Function to compute the evaluation metric
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model and tokenizer
trainer.save_model("./QuestionableYouTubeVideoTitleV1_finetuned")  # Save the model
tokenizer.save_pretrained("./QuestionableYouTubeVideoTitleV1_finetuned")  # Save the tokenizer


c:\Users\Edwar\OneDrive - Calvin Institute of Technology\Documents\CIT\Semester 9\IEE3022 Deep Learning-MSI\UAS\UAS debugging\.venv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\Edwar\OneDrive - Calvin Institute of Technology\Documents\CIT\Semester 9\IEE3022 Deep Learning-MSI\UAS\UAS debugging\.venv\Lib\site-packages\torch\nn\parallel\data_parallel.py:37: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(


  0%|          | 0/840 [00:00<?, ?it/s]

c:\Users\Edwar\OneDrive - Calvin Institute of Technology\Documents\CIT\Semester 9\IEE3022 Deep Learning-MSI\UAS\UAS debugging\.venv\Lib\site-packages\torch\nn\parallel\_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
c:\Users\Edwar\OneDrive - Calvin Institute of Technology\Documents\CIT\Semester 9\IEE3022 Deep Learning-MSI\UAS\UAS debugging\.venv\Lib\site-packages\torch\cuda\nccl.py:16: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn("PyTorch is not compiled with NCCL support")


  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 2.1021509170532227, 'eval_bleu': 0, 'eval_runtime': 0.9795, 'eval_samples_per_second': 56.152, 'eval_steps_per_second': 4.084, 'epoch': 1.0}


...

  0%|          | 0/4 [00:00<?, ?it/s]

{'eval_loss': 1.8969427347183228, 'eval_bleu': 0, 'eval_runtime': 1.2903, 'eval_samples_per_second': 42.626, 'eval_steps_per_second': 3.1, 'epoch': 30.0}


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


{'train_runtime': 495.8701, 'train_samples_per_second': 13.189, 'train_steps_per_second': 1.694, 'train_loss': 0.29044576372419084, 'epoch': 30.0}


('./fine_tuned_gpt2_ver1.17\\tokenizer_config.json',
 './fine_tuned_gpt2_ver1.17\\special_tokens_map.json',
 './fine_tuned_gpt2_ver1.17\\vocab.json',
 './fine_tuned_gpt2_ver1.17\\merges.txt',
 './fine_tuned_gpt2_ver1.17\\added_tokens.json',
 './fine_tuned_gpt2_ver1.17\\tokenizer.json')

Prompt here